In [1]:
import qutip as qt
import numpy as np
import circuit
import circuit_block
import stabilizer

# Demo 1: Creating a error model

In order to simulate the surface code in a network we must first simulate the process of creating the network itself. That is connecting the nodes through entanglement. Following the theory here we will use GHZ states to perform the required stabilizer measurements. Since we are considering certain error models those stabilizer measurements will have an intrinsic error every time they are made. In this demo we will go through all the steps into obtaining the error models of such noisy stabilizers.

The outline of the procedure is as follows:
1. Set the error rates for the operations and the environment.
2. Choose a protocol for creating a GHZ state among the nodes.
3. Run the selected protocol n times to obtain an average GHZ state and a average generation time
4. Model the stabilizer measurement as a channel and use the Choi-Jamiolkowsky isomorphism to obtain the $\chi$-matrix in the Pauli basis.

After this we are left with a dictionary which contains the error rates for every possible error induced during the noisy stabilizer measurement. This together with the time needed to create the GHZ states will be used to simulate the surface code.

### STEP 1: Set the error rates
Following the error models for NV centers here we set the values for each parameter

In [38]:
# Set parameters
# Gate and measurement error rates
ps = 0.003
pm = 0.003
pg = 0.003

# Very optimistic enviromental error rate and entanglement generation rate
a0 = 1.5
a1 = 1/80.
eta = 1/100.

# Theta to optimize entanglement generation
theta = .24

### STEP 2: Choose a protocol for generating a GHZ state
We need to define a circuit to generate a GHZ between 4 nodes.
We choose the following circuit:
<img src="extras/ghz_creation.png" width="800">
<img src="extras/EPL.png" width="800">

Now let's see how to make this in code.
See the files **circuit.py** and **circuit_block.py**

In [39]:
# Initialize the circuit block and stabilizer objects 
# Circuits are assemebled in reversed order due to recursion
cb = circuit_block.Blocks(ps, pm, pg, eta, a0, a1, theta)
epl = circuit.Circuit(a0=a0, a1=a1,
                          circuit_block=cb.start_epl)

# Phase 3 - Create GHZ
# Perform the measurements
protocol = circuit.Circuit(a0=a0, a1=a1,
                           circuit_block=cb.collapse_ancillas_GHZ,
                           ghz_size=4,
                           measure_pos=[4, 5, 6, 7])
# Apply two qubit gates in the nodes
protocol.add_circuit(circuit_block=cb.two_qubit_gates, controls=[1, 3, 0, 2],
                     targets=[4, 5, 6, 7], sigma="X")

# Phase 2 Create last two pairs
pair = circuit.Circuit(a0=a0, a1=a1, circuit_block=cb.swap_pair,
                       pair=[0, 1])
pair.add_circuit(circuit_block=cb.start_epl)
# Wrapper to run in parallel
wrap_EPL_parallel = circuit.Circuit(a0=a0, a1=a1,
                                    circuit_block=pair.run_parallel)

protocol.add_circuit(circuit_block=wrap_EPL_parallel.append_circuit)

# Phase 1 Create initial two pairs

protocol.add_circuit(circuit_block=epl.run_parallel)

### Result
The code returns a complete circuit that we will need to run each time to generate the a Bell pair each time. 

In [40]:
ghz, resources = protocol.run()

# Reference GHZ state to compare the result
ghz_ref = qt.ghz_state(4) * qt.ghz_state(4).dag()

print("Resulting state fidelity: ", qt.fidelity(ghz, ghz_ref))
print("Resources used: ", resources)

Resulting state fidelity:  0.9847724039848701
Resources used:  Counter({'two_qubit_gate': 44, 'bell_pair': 20, 'single_qubit_gate': 11, 'measurement': 11, 'time': 0.11578509999999999, 'time0': 0.104784, 'time1': 0.011001100000000002})


Since there is a Monte Carlo simulation running under the hood if we run the circuit two times we will obtain different results.


For improved results perform multiple runs and average over them. For safe statistics a good number of iterations is $\approx$ 2000. 

In [41]:
# Lists to save fidelities and times
fidelity = []
times = []

iterations = 5
rho = ghz * 0
for i in range(iterations):
    r, c = protocol.run()

    # Twirl the resulting state to distribute any error symmetrically over the qubits
    r = stab.twirl_ghz(r)
    
    fidelity += [qt.fidelity(r, ghz_ref)]
    rho += r
    times += [c["time"]]
    
rho = rho/iterations
print("Fidelity: ", np.average(fidelity), np.std(fidelity))
print("Time:", np.average(times), np.std(times))


Fidelity:  0.9862042643175443 0.001413069194952346
Time: 0.07722621999999998 0.02737438853566596


### Other protocols
As we can see the chosen protocol is not very effective. The good news is that there are better ones! For example using the protocols that make use of Barret-Kok for generating entanglement rather than EPL are much more effective. 

For all the protocols available see the file **protocols.py**

## Extracting the error model out of a GHZ state
Now that we have created a GHZ state through the nodes we need to extract the error rates obtained when making a stabilizer measurement. To achieve this we need first to think making a stabilizer measurement as quantum channel, form there we can use the Choi-Jamiolkowsky isomorphism to extract the $\chi$-matrix which contains all the information about the channel

First define which stabilizer (star or plaquette) is going to be evaluated and initialize the required objects.

In [42]:
import noise_modeling

# Select a type of stabilizer to extract the model from
stab_type = "X"
stab_size = 4

# Initialize objects
model = noise_modeling.NoiseModel(stab_size, stab_type)
model.separate_basis_parity()

# Choi state for noise noise modeling
choi = model._choi_state_ket(stab_size)
choi = choi * choi.dag()
targets = list(range(stab_size))


### Stabilizer as a quantum channel

$$\mathcal{S}(\rho) = p_{e}\rho^{even} \otimes \lvert0\rangle \langle0\rvert + p_{o}\rho^{odd} \otimes \lvert1\rangle \langle1\rvert$$

In code we use the functions in **stabilizer.py**

In [43]:
probs, rhos = stab.measure_ghz_stabilizer(choi, ghz, targets, stab_type)

Express the $\chi$-matrix in the Pauli basis. See the code in **noise_modeling.py**.

In [44]:
# Set channel output and make chi matrix
model.set_rho(rhos, probs)
model.make_chi_matrix()
    
# Sanity check: Check the total sum of the decomposition = 1
print("Sum of all probabilties: ", model.check_total_sum())

print("Decomposition: ", model.chi)


Sum of all probabilties:  0.9999999996698073
Decomposition:  {'IIII_OK': 0.9273072442432951, 'IIII_NOK': 0.0460805880300378, 'IIXI_OK': 0.009943485874529031, 'IIXI_NOK': 0.009943485874529031, 'IIIY_OK': 0.001567884018536117, 'IIIY_NOK': 0.001567884018536117, 'IZII_OK': 0.0015678839088475419, 'IZII_NOK': 0.001567883908847564, 'XIIX_OK': 0.00018622804035471315, 'XIIX_NOK': 0.00015431325106153473, 'XIYI_OK': 2.4447847787113913e-05, 'XIYI_NOK': 2.4447847787113913e-05, 'ZIXI_OK': 2.444804879958313e-05, 'ZIXI_NOK': 2.444804879958313e-05, 'IYIY_OK': 1.894315057846557e-06, 'IYIY_NOK': 1.894315057846557e-06, 'YIIZ_OK': 3.7885361168933444e-06, 'YIIZ_NOK': 3.7885361168933444e-06, 'ZZII_OK': 1.8942279505778804e-06, 'ZZII_NOK': 1.8942279506002672e-06, 'ZYIX_OK': 3.9127507649317587e-08, 'ZYIX_NOK': 3.912750764931639e-08, 'XIZZ_OK': 3.908104093536989e-08, 'XIZZ_NOK': 3.908104093536989e-08, 'YYIY_OK': 1.0135465428331273e-09, 'YYIY_NOK': 1.0135465428331273e-09, 'YZIY_OK': 3.034362818027057e-09, 'YZIY_N

In order to gain an idea of the error rates given by the noisy stabilizer measurement 
we can look at some of the error rates obtained.

For no error we look at $IIII\_OK$ and only measurement error $IIII\_NOK$. Qubit errors are more complex since we have the rates for all the possible combinations of error. Nonetheless we can have an idea by looking at the value
of $E=(1 -IIII\_OK - IIII\_NOK)/4$. 

For comparison remember the thresholds for the surface code for a qubit error rate of $q$ and a measurement error rate of $p$ are $q=p=0.01$

In [45]:
    I_OK = model.chi["IIII_OK"]
    I_NOK = model.chi["IIII_NOK"]
    # The sum of all physical errors
    E = (1 - model.chi["IIII_OK"] - model.chi["IIII_NOK"])/4.
    print("Errors:")
    print("OK: ", I_OK)
    print("NOK: ", I_NOK)
    print("QUBIT ERROR: ", E)

Errors:
OK:  0.9273072442432951
NOK:  0.0460805880300378
QUBIT ERROR:  0.0066530419316667665


Finally, in order to use this error model in the surface code simulations we need to save this dictionary. We use pickle for this end.

In [46]:
import tools.names as names
import pickle


file_name = names.chi(ps, pm, pg, eta, a0, a1, theta,
                      stab_size, stab_type, "DEMO")

print("File name: ", file_name)
pickle_out = open(file_name, "wb")
pickle.dump(model.chi, pickle_out, protocol=2)
pickle_out.close()

File name:  /home/eduardo/thesis/fault_tolerance/data/CHI_DEMO_X_4_ps=0.003_pm=0.003_pg=0.003_eta=0.01_a0=1.5_a1=0.0125_theta=0.24.dict
